## 1. Introduction
Discuss the business problem and audience who would interested

I plan to start a Cafe in a multicultural city and would like to do some background research on the best location for the cafe.
The research should be able to provide me answers to the following questions:
1. What types of cafe are popular now in New York City & Singapore?
2. Why are these type of cafe popular?
3. What are the types of businesses around these popular cafes?

## 2. Data
Describe the data that you will be using to solve the problem or execute your idea. 

1. New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude

   (Data Source: https://cocl.us/new_york_dataset)


2. Singapore City data (Specifically Tanjong Pagar Area) that contains Towns, Neighborhoods along with their latitude and longitude.

   (Data Source:  Foursquare API)

Tanjong Pagar is a multicultural district in Singapore with international dining and diverse nightlife options. Craig Road and Duxton Hill have a mix of bars and global restaurants, while Maxwell Road Food Centre is known for street food such as noodles and chicken rice. “Little Korea”, at the northern end of Tanjong Pagar Road, has casual eateries and specialist grocery stores. Nearby, Everton Park is known for its hip cafes. Therefore Tanjong Pagar is being selected to be the district to study.
   
   

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'UCVO2GVTL12OH0OWGRITBRVBUO2XABHYHFTVA2YIYXWZY2XE' # your Foursquare ID
CLIENT_SECRET = 'C5HRVGKZXEGWYOV0D03E4HRRCM5JW2AJ44A3ORJXDLSFLFUO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UCVO2GVTL12OH0OWGRITBRVBUO2XABHYHFTVA2YIYXWZY2XE
CLIENT_SECRET:C5HRVGKZXEGWYOV0D03E4HRRCM5JW2AJ44A3ORJXDLSFLFUO


### Geocoder

In [15]:
address = 'Tanjong Pagar, Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.2764189, 103.8429295.


In [16]:
search_query = 'Cafe'
radius = 500
print(search_query + ' .... OK!')

Cafe .... OK!


In [17]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UCVO2GVTL12OH0OWGRITBRVBUO2XABHYHFTVA2YIYXWZY2XE&client_secret=C5HRVGKZXEGWYOV0D03E4HRRCM5JW2AJ44A3ORJXDLSFLFUO&ll=1.2764189,103.8429295&v=20180604&query=Cafe&radius=500&limit=30'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ceab98f6a6071493f71b2e0'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'primary': True,
      'shortName': 'American'}],
    'hasPerk': False,
    'id': '4b610addf964a52081072ae3',
    'location': {'address': 'Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong Pagar Road',
     'cc': 'SG',
     'city': 'Singapore',
     'country': 'Singapore',
     'distance': 82,
     'formattedAddress': ['Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong Pagar Road',
      '081007',
      'Singapore'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 1.2768220777373407,
       'lng': 103.8435486473012}],
     'lat': 1.2768220777373407,
     'lng': 103.8435486473012,
     'postalCode': '081007'},
    'name': 'Bluemist Cafe Bistro',
    're

In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'pluralName': 'American Restaurants', 'short...",False,4b610addf964a52081072ae3,"Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong...",SG,Singapore,Singapore,NaN,82,"[Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjon...","[{'lat': 1.2768220777373407, 'label': 'display...",1.276822,103.843549,NaN,081007,NaN,Bluemist Cafe Bistro,v-1558886799,NaN
1,"[{'pluralName': 'Nightclubs', 'shortName': 'Ni...",False,4b0d52c7f964a520504623e3,65 Neil Rd.,SG,Singapore,Singapore,NaN,342,"[65 Neil Rd., 088897, Singapore]","[{'lat': 1.2794357645685264, 'label': 'display...",1.279436,103.842319,NaN,088897,NaN,Taboo Café & Bar,v-1558886799,NaN
2,"[{'pluralName': 'Asian Restaurants', 'shortNam...",False,4d58bc313281b1f7b468b32f,7 Craig Rd.,SG,Singapore,Singapore,NaN,118,"[7 Craig Rd., 089667, Singapore]","[{'lat': 1.2774705411789302, 'label': 'display...",1.277471,103.843107,NaN,089667,NaN,Golden Cafe Eating House,v-1558886799,NaN
3,"[{'pluralName': 'Restaurants', 'shortName': 'R...",False,4b84e425f964a520c64631e3,114 Tanjong Pagar Road,SG,Singapore,Singapore,NaN,177,"[114 Tanjong Pagar Road, 088529, Singapore]","[{'lat': 1.2778405205280887, 'label': 'display...",1.277841,103.843649,NaN,088529,NaN,New Harbour Cafe & Bar,v-1558886799,NaN
4,"[{'pluralName': 'Cafés', 'shortName': 'Café', ...",False,4b987225f964a520954235e3,M Hotel,SG,Singapore,Singapore,81 Anson Rd.,347,"[M Hotel (81 Anson Rd.), 079908, Singapore]","[{'lat': 1.2739816398549644, 'label': 'display...",1.273982,103.844885,NaN,079908,NaN,Cafe 2000,v-1558886799,NaN


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Bluemist Cafe Bistro,American Restaurant,"Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjong...",SG,Singapore,Singapore,NaN,82,"[Blk 7 Tanjong Pagar Plaza, #01-103 165 Tanjon...","[{'lat': 1.2768220777373407, 'label': 'display...",1.276822,103.843549,NaN,081007,NaN,4b610addf964a52081072ae3
1,Taboo Café & Bar,Nightclub,65 Neil Rd.,SG,Singapore,Singapore,NaN,342,"[65 Neil Rd., 088897, Singapore]","[{'lat': 1.2794357645685264, 'label': 'display...",1.279436,103.842319,NaN,088897,NaN,4b0d52c7f964a520504623e3
2,Golden Cafe Eating House,Asian Restaurant,7 Craig Rd.,SG,Singapore,Singapore,NaN,118,"[7 Craig Rd., 089667, Singapore]","[{'lat': 1.2774705411789302, 'label': 'display...",1.277471,103.843107,NaN,089667,NaN,4d58bc313281b1f7b468b32f
3,New Harbour Cafe & Bar,Restaurant,114 Tanjong Pagar Road,SG,Singapore,Singapore,NaN,177,"[114 Tanjong Pagar Road, 088529, Singapore]","[{'lat': 1.2778405205280887, 'label': 'display...",1.277841,103.843649,NaN,088529,NaN,4b84e425f964a520c64631e3
4,Cafe 2000,Café,M Hotel,SG,Singapore,Singapore,81 Anson Rd.,347,"[M Hotel (81 Anson Rd.), 079908, Singapore]","[{'lat': 1.2739816398549644, 'label': 'display...",1.273982,103.844885,NaN,079908,NaN,4b987225f964a520954235e3
5,Han's Cafe - Tanjong Pagar Center,Café,7 Wallich Street,SG,NaN,Singapore,NaN,236,"[7 Wallich Street, 078879, Singapore]","[{'lat': 1.2767379869213256, 'label': 'display...",1.276738,103.845032,NaN,078879,NaN,58575b058f0be459e4487256
6,Imperial Treasure Bakery & Cafe 御宝面包之窗,Bakery,#01-10/14A 100AM Mall,SG,Singapore,Singapore,100 Tras St,188,"[#01-10/14A 100AM Mall (100 Tras St), 079027, ...","[{'lat': 1.274762105690608, 'label': 'display'...",1.274762,103.843292,Financial District,079027,NaN,50d115bfe4b072dcb75a475b
7,McDonald's / McCafé,Fast Food Restaurant,"#01-03, Springleaf Tower",SG,Singapore,Singapore,3 Anson Rd,378,"[#01-03, Springleaf Tower (3 Anson Rd), 079909...","[{'lat': 1.2748651681501353, 'label': 'display...",1.274865,103.845952,Financial District,079909,NaN,4b57df11f964a520984428e3
8,Cuscaden Cafe & Pub,Pub,18 Duxton Rd,SG,Singapore,Singapore,NaN,349,"[18 Duxton Rd, Singapore]","[{'lat': 1.2795303917866119, 'label': 'display...",1.279530,103.843319,NaN,NaN,NaN,4d1b587883c5a1cdc5132b4e
9,Huggs Cafe,Café,GB Building,SG,Singapore,Singapore,143 Cecil Street,611,"[GB Building (143 Cecil Street), Singapore]","[{'lat': 1.2796347302621813, 'label': 'display...",1.279635,103.847385,NaN,NaN,NaN,4ca9660714c337042beddd3b


In [22]:
dataframe_filtered.name

0                       Bluemist Cafe Bistro
1                           Taboo Café & Bar
2                   Golden Cafe Eating House
3                     New Harbour Cafe & Bar
4                                  Cafe 2000
5          Han's Cafe - Tanjong Pagar Center
6     Imperial Treasure Bakery & Cafe 御宝面包之窗
7                        McDonald's / McCafé
8                        Cuscaden Cafe & Pub
9                                 Huggs Cafe
10                              Turkish Cafe
11                                 Wang Cafe
12                              Cafe Tartufo
13        Cafe Ergo Sum | Smoke Shack Duxton
14                         The Showroom Cafe
15                           好好食堂 2Good Cafe
16         Bakerz Cafe @ International Plaza
17                            Open Book Cafe
18                    Dennis Cafe Restaurant
19                       Pacific Allied Cafe
20                Seong Huat Street Art Cafe
21                Bingki Korean Dessert Cafe
22        